# Projeto Imersão IA Alura + Google Gemini

---

## Tema: Assistente de Organização Financeira Pessoal
## Objetivo: Analisar a vida financeira do cliente e traçar estratégias para sua melhor gestão.

---



In [1]:
!pip -q install google-genai



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\arthu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
# Configurar a API Key do Google Gemini - Salvando na memória do Collab

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Configurar o cliente SDK que vai receber as informações do Google Gemini

from google import genai
client = genai.Client()

Model_ID = "gemini-2.0-flash"

In [ ]:
from IPython.display import HTML, Markdown


In [ ]:
!pip -q install google.adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

    # Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
pip install -q -U google-generativeai python-dotenv

In [ ]:
!pip install -q -U gspread oauth2client

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# -*- coding: utf-8 -*-
"""
Script Python para coletar dados, realizar análise com IA Generativa,
planejar a reserva de emergência, gerar um painel visual no Google Sheets
e simular acompanhamento e reforço de comportamento financeiro.
(Etapa 1, 2, 4, 5 e 3).
Atualizado para usar gemini-2.0-flash, prompt refinado e incluir Etapa 5.
Corrigido o erro NameError no método __str__.
"""

import sys
import os
import google.generativeai as genai
from dotenv import load_dotenv
import gspread # Biblioteca para interagir com Google Sheets API
from oauth2client.service_account import ServiceAccountCredentials # Para autenticação

# --- IMPORTANTE PARA GOOGLE COLAB ---
# Execute a célula abaixo no seu notebook Colab ANTES deste código:
# from google.colab import drive
# drive.mount('/content/drive')
# ------------------------------------


# Carrega as variáveis de ambiente (se estiver usando um .env para a chave da API GenAI)
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    print("ERRO: A variável de ambiente GOOGLE_API_KEY não está configurada.")
    print("Certifique-se de que ela está no seu arquivo .env (se estiver usando) ou definida no ambiente Colab.")
    sys.exit(1)

# --- Configuração do Modelo Gemini ---
# Usando o modelo gemini-2.0-flash conforme solicitado.
genai.configure(api_key=GOOGLE_API_KEY)
MODEL = genai.GenerativeModel('gemini-2.0-flash') # <--- Modelo atualizado

# --- Configuração do Google Sheets API ---
# !!! ATUALIZE ESTE CAMINHO COM O CAMINHO COMPLETO PARA O ARQUIVO JSON NO SEU GOOGLE DRIVE !!!
# EXEMPLO CORRETO: '/content/drive/MyDrive/Colab Notebooks/Imersão Alura/nome_do_seu_arquivo_credenciais.json'
# O CAMINHO ABAIXO É APENAS UM PLACEHOLDER. VOCÊ DEVE SUBSTITUÍ-LO!
SERVICE_ACCOUNT_KEY_FILE = '/content/drive/MyDrive/Colab Notebooks/Imersão Alura/google_sheets_key.json' # <--- ATUALIZE AQUI COM O CAMINHO COMPLETO DO ARQUIVO JSON

# Escopos de permissão necessários. Permite ler e escrever em planilhas e gerenciar arquivos no Drive (incluindo compartilhar).
SCOPE = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

# Autenticação
try:
    # Verifica se o arquivo de credenciais existe no caminho especificado
    if not os.path.exists(SERVICE_ACCOUNT_KEY_FILE):
         raise FileNotFoundError(f"Arquivo de credenciais não encontrado: {SERVICE_ACCOUNT_KEY_FILE}")

    CREDS = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_KEY_FILE, SCOPE)
    CLIENT = gspread.authorize(CREDS)
    #print("Autenticação com Google Sheets API bem-sucedida.")
except FileNotFoundError as e:
    print(f"ERRO: {e}")
    print("Por favor, verifique se o caminho para o arquivo JSON está correto após montar seu Google Drive no Colab.")
    print(f"O caminho configurado é: '{SERVICE_ACCOUNT_KEY_FILE}'")
    sys.exit(1)
except Exception as e:
    print(f"ERRO de autenticação com Google Sheets API: {e}")
    print(f"Detalhes: {e}")
    print(f"Verifique se o arquivo JSON não está corrompido e se a conta de serviço está ativa.")
    sys.exit(1)


# --- Estrutura de Dados do Usuário ---
class DadosFinanceirosUsuario:
    def __init__(self):
        self.email_pessoal_usuario = ""
        self.renda_liquida_mensal = 0.0
        self.fonte_renda = ""
        self.gastos_por_categoria = {}
        self.patrimonio = {}
        self.dividas = []
        self.dependentes = 0
        self.estabilidade_financeira = ""

        # Resultados da análise numérica (Etapa 2)
        self.total_gastos_mensais = 0.0
        self.total_gastos_essenciais = 0.0
        self.classificacao_gastos = {}
        self.status_fluxo_caixa = ""
        self.saldo_mensal = 0.0

        # Resultados do planejamento da reserva (Etapa 4)
        self.valor_reserva_emergencia_ideal = 0.0
        self.meses_reserva_sugerido = 0
        self.saldo_mensal_para_reserva = 0.0
        self.tempo_para_montar_reserva_meses = 0.0
        self.meta_mensal_reserva = 0.0

        # Resultados do acompanhamento e reforço (Etapa 5)
        self.relatorio_mensal_simulado = "" # Simula o conteúdo do relatório
        self.feedback_ia_comportamento = "" # Feedback e recomendações da IA sobre comportamento


    def __str__(self):
        """
        Representação em string dos dados coletados e resultados da análise.
        Corrigido NameError usando 'self' para acessar atributos da instância.
        """
        resumo = (f"--- Resumo dos Dados Coletados e Análise ---\n"
                  f"Email do Usuário: {self.email_pessoal_usuario}\n"
                  f"Renda Líquida Mensal: R$ {self.renda_liquida_mensal:.2f}\n"
                  f"Total de Gastos Mensais: R$ {self.total_gastos_mensais:.2f}\n"
                  f"Total de Gastos Essenciais: R$ {self.total_gastos_essenciais:.2f}\n"
                  f"Saldo Mensal: R$ {self.saldo_mensal:.2f}\n"
                  f"Status do Fluxo de Caixa: {self.status_fluxo_caixa}\n"
                  f"--- Planejamento da Reserva de Emergência ---\n"
                  f"Valor Ideal da Reserva: R$ {self.valor_reserva_emergencia_ideal:.2f} ({self.meses_reserva_sugerido} meses de gastos essenciais)\n"
                  f"Saldo Mensal Disponível para Reserva: R$ {self.saldo_mensal_para_reserva:.2f}\n"
                  # --- CORRIGIDO: Usando 'self' para acessar o atributo ---
                  f"Tempo Estimado para Montar a Reserva: {self.tempo_para_montar_reserva_meses:.1f} meses\n"
                  # --- CORRIGIDO: Usando 'self' para acessar o atributo ---
                  f"Meta Mensal Sugerida para Reserva: R$ {self.meta_mensal_reserva:.2f}\n"
                  f"--- Relatório Mensal Simulado ---\n{self.relatorio_mensal_simulado}\n"
                  f"--- Análise e Recomendações da IA (Comportamento) ---\n{self.feedback_ia_comportamento}\n"
                  f"---------------------------------"
                  )
        return resumo


# --- Funções de Coleta com Validação Básica (Mantidas) ---
# Inclua aqui as funções get_float_input, get_int_input, get_string_input,
# coletar_gastos_por_categoria, coletar_patrimonio, coletar_dividas do script anterior.

def get_float_input(prompt):
    """Coleta um valor float do usuário com validação básica."""
    while True:
        try:
            valor = float(input(prompt).replace(',', '.')) # Permite vírgula ou ponto decimal
            if valor < 0:
                 print("Valor não pode ser negativo. Tente novamente.")
                 continue
            return valor
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")

def get_int_input(prompt):
    """Coleta um valor inteiro do usuário com validação básica."""
    while True:
        try:
            valor = int(input(prompt))
            if valor < 0:
                print("Valor não pode ser negativo. Tente novamente.")
                continue
            return valor
        except ValueError:
            print("Entrada inválida. Por favor, digite um número inteiro.")

def get_string_input(prompt):
    """Coleta uma string não vazia do usuário."""
    while True:
        valor = input(prompt).strip()
        if valor:
            return valor
        else:
            print("Entrada não pode ser vazia. Tente novamente.")

def coletar_gastos_por_categoria():
    """Coleta os gastos mensais por categoria, com classificação inicial."""
    gastos = {}
    classificacao_map = {} # Dicionário para armazenar classificações por categoria
    print("\n--- Gastos Mensais por Categoria ---")
    print("Digite o valor gasto para cada categoria. Digite 'fim' quando terminar.")

    categorias_sugeridas = {
        'Moradia (Aluguel)': {'tipo': 'Essencial', 'natureza': 'Fixo'},
        'Alimentação': {'tipo': 'Essencial', 'natureza': 'Variável'},
        'Transporte': {'tipo': 'Essencial', 'natureza': 'Variável'},
        'Saúde': {'tipo': 'Essencial', 'natureza': 'Variável'},
        'Educação': {'tipo': 'Essencial', 'natureza': 'Fixo'},
        'Lazer': {'tipo': 'Supérfluo', 'natureza': 'Variável'},
        'Assinaturas': {'tipo': 'Supérfluo', 'natureza': 'Fixo'},
        'Contas de Consumo (água, luz, gás)': {'tipo': 'Essencial', 'natureza': 'Variável'},
        #'Outros': {'tipo': 'Variável', 'natureza': 'Variável'}
    }

    for categoria, classificacao in categorias_sugeridas.items():
        prompt = f"Gasto com {categoria} (R$): "
        valor = get_float_input(prompt)
        if valor > 0:
            gastos[categoria] = valor
            classificacao_map[categoria] = classificacao

    while True:
        categoria_extra = get_string_input("Adicionar outra categoria de gasto? (ou 'fim'): ").capitalize()
        if categoria_extra.lower() == 'fim':
            break
        if categoria_extra in gastos:
            print(f"Categoria '{categoria_extra}' já adicionada. Digite outro nome.")
            continue

        print(f"Classifique o gasto '{categoria_extra}':")
        tipo = get_string_input("É 'Essencial' ou 'Supérfluo'? ").capitalize()
        natureza = get_string_input("É 'Fixo' ou 'Variável'? ").capitalize()

        prompt = f"Gasto com {categoria_extra} (R$): "
        valor = get_float_input(prompt)
        if valor > 0:
            gastos[categoria_extra] = valor
            classificacao_map[categoria_extra] = {'tipo': tipo, 'natureza': natureza}

    return gastos, classificacao_map

def coletar_patrimonio():
    """Coleta a situação patrimonial do usuário."""
    patrimonio = {}
    print("\n--- Situação Patrimonial ---")
    print("Informe o valor estimado de seus bens. Digite 'fim' quando terminar.")

    bens_sugeridos = ['Imóveis', 'Veículos', 'Investimentos', 'Contas Bancárias (saldo total)', 'Outros Ativos']

    for bem in bens_sugeridos:
        prompt = f"Valor de {bem} (R$): "
        valor = get_float_input(prompt)
        if valor > 0:
            patrimonio[bem] = valor

    while True:
        bem_extra = get_string_input("Adicionar outro bem/ativo? (ou 'fim'): ").capitalize()
        if bem_extra.lower() == 'fim':
            break
        if bem_extra in patrimonio:
            print(f"Bem/ativo '{bem_extra}' já adicionado. Digite outro nome.")
            continue
        prompt = f"Valor de {bem_extra} (R$): "
        valor = get_float_input(prompt)
        if valor > 0:
            patrimonio[bem_extra] = valor

    return patrimonio

def coletar_dividas():
    """Coleta as dívidas existentes do usuário."""
    dividas = []
    print("\n--- Dívidas Existentes ---")
    print("Informe os detalhes de cada dívida. Digite 'fim' no tipo de dívida quando terminar.")

    while True:
        print("-" * 20)
        tipo_divida = get_string_input("Tipo da dívida (Ex: Cartão, Empréstimo, Financiamento) (ou 'fim'): ").capitalize()
        if tipo_divida.lower() == 'fim':
            break

        valor_original = get_float_input(f"Valor total original da dívida de {tipo_divida} (R$): ")
        valor_restante = get_float_input(f"Valor restante a pagar da dívida de {tipo_divida} (R$): ")
        taxa_juros_anual = get_float_input(f"Taxa de juros ANUAL da dívida de {tipo_divida} (Ex: 0.15 para 15%): ")
        parcelas_totais = get_int_input(f"Número TOTAL de parcelas da dívida de {tipo_divida}: ")
        parcelas_restantes = get_int_input(f"Número de parcelas RESTANTES da dívida de {tipo_divida}: ")

        dividas.append({
            'tipo': tipo_divida,
            'valor_original': valor_original,
            'valor_restante': valor_restante,
            'taxa_juros_anual': taxa_juros_anual,
            'parcelas_totais': parcelas_totais,
            'parcelas_restantes': parcelas_restantes
        })
        print("Dívida adicionada.")

    return dividas


# --- Funções de Análise (Etapa 2) ---

def gerar_analise_ia(dados_usuario):
    """
    Usa um modelo de IA Generativa para fornecer uma análise qualitativa da situação financeira,
    incluindo feedback de comportamento.
    O prompt foi atualizado para a Etapa 5.
    """
    print("\n--- Gerando Análise Inteligente da IA (aguarde) ---")

    # --- PROMPT ATUALIZADO PARA ETAPA 5 ---
    prompt_parts = [
        """Você é um consultor financeiro altamente experiente, com mais de 7 anos de experiência de mercado, e empático. Sua tarefa é analisar a situação financeira de um indivíduo e fornecer um diagnóstico claro, personalizado
         e acionável em português do Brasil. Com base nos dados fornecidos, identifique os principais pontos fortes e fracos, os maiores desafios e as oportunidades. Ofereça sugestões gerais para os próximos passos,
         incentivando o usuário a tomar ações positivas. Seja direto, mas compreensivo. Use linguagem fácil de entender, evitando jargões excessivos, mas mantendo a profundidade da análise e procure não enrolar muito,
         procure ser sucinto . Não use markdown no output.

         Inclua feedback sobre o progresso em relação à reserva de emergência (se aplicável) e sugestões concretas para otimizar gastos ou acelerar a quitação de dívidas, se necessário. Baseie as sugestões nos dados de gastos fornecidos.
         Exemplos de sugestões: "Considere reduzir seus gastos com Lazer em X R$/mês", "Priorize a quitação da dívida de [Tipo da Dívida]".
         Se o usuário estiver em superávit e no caminho para a reserva, reforce o comportamento positivo e sugira manter o foco.

         Mas se o usuário estiver com a conta no vermelho indique também caminhos e estrategia que ele pode adotar para melhorar seu saldo, como também dicas de renda extra (algo simples e que grande parte das pessoas poderia fazer
         ou algo relacionado a sua área de atuação a qual o usuário pertence), para ter uma alternativa a mais e melhorar sua renda.

         ---

         """, # <--- Instruções adicionadas para Etapa 5
        "\n--- Dados Financeiros do Usuário ---",
        f"Renda Líquida Mensal: R$ {dados_usuario.renda_liquida_mensal:.2f}",
        f"Fonte de Renda: {dados_usuario.fonte_renda}",
        f"Dependentes: {dados_usuario.dependentes}",
        f"Estabilidade Financeira: {dados_usuario.estabilidade_financeira}",
        "\nGastos por Categoria:",
    ]
    for categoria, valor in dados_usuario.gastos_por_categoria.items():
        classif = dados_usuario.classificacao_gastos.get(categoria, {'tipo': 'N/A', 'natureza': 'N/A'})
        prompt_parts.append(f"- {categoria}: R$ {valor:.2f} (Tipo: {classif['tipo']}, Natureza: {classif['natureza']})")

    prompt_parts.append("\nSituação Patrimonial:")
    if not dados_usuario.patrimonio:
        prompt_parts.append("- Nenhum patrimônio informado.")
    else:
        for bem, valor in dados_usuario.patrimonio.items():
            prompt_parts.append(f"- {bem}: R$ {valor:.2f}")

    prompt_parts.append("\nDívidas Existentes:")
    if not dados_usuario.dividas:
        prompt_parts.append("- Nenhuma dívida informada.")
    else:
        for divida in dados_usuario.dividas:
            prompt_parts.append(f"- Tipo: {divida['tipo']}, Valor Restante: R$ {divida['valor_restante']:.2f}, Taxa Juros Anual: {divida['taxa_juros_anual']:.2%}, Parcelas Restantes: {divida['parcelas_restantes']}/{divida['parcelas_totais']}")

    prompt_parts.append(f"\n--- Resumo Numérico (Já Calculado) ---")
    prompt_parts.append(f"Total de Gastos Mensais: R$ {dados_usuario.total_gastos_mensais:.2f}")
    prompt_parts.append(f"Total de Gastos Essenciais: R$ {dados_usuario.total_gastos_essenciais:.2f}")
    prompt_parts.append(f"Saldo Mensal (Renda - Gastos): R$ {dados_usuario.saldo_mensal:.2f}")
    prompt_parts.append(f"Status do Fluxo de Caixa: {dados_usuario.status_fluxo_caixa}")

    # Adiciona informações da reserva ao prompt para a IA comentar
    prompt_parts.append(f"\n--- Planejamento da Reserva de Emergência (Já Calculado) ---")
    prompt_parts.append(f"Valor Ideal da Reserva de Emergência: R$ {dados_usuario.valor_reserva_emergencia_ideal:.2f} ({dados_usuario.meses_reserva_sugerido} meses de gastos essenciais)")
    prompt_parts.append(f"Saldo Mensal Disponível para Construir a Reserva: R$ {dados_usuario.saldo_mensal_para_reserva:.2f}")
    prompt_parts.append(f"Tempo Estimado para Montar a Reserva (se possível): {dados_usuario.tempo_para_montar_reserva_meses:.1f} meses")
    prompt_parts.append(f"Meta Mensal Sugerida para Reserva: R$ {dados_usuario.meta_mensal_reserva:.2f}")

    # Adiciona informações de progresso (SIMULADO para esta iteração)
    # Em uma versão real, você passaria o valor atual da reserva.
    # Aqui, a IA terá que inferir com base no saldo e meta.
    prompt_parts.append("\n--- Progresso Atual (Baseado no Saldo Mensal e Meta) ---")
    if dados_usuario.saldo_mensal > 0 and dados_usuario.meta_mensal_reserva > 0:
        prompt_parts.append(f"Com o saldo atual de R$ {dados_usuario.saldo_mensal:.2f} e meta mensal de R$ {dados_usuario.meta_mensal_reserva:.2f}, você está no caminho para construir a reserva.")
        # Poderíamos adicionar aqui: "Você já acumulou R$ X na reserva." se tivéssemos o dado.
    elif dados_usuario.saldo_mensal <= 0:
         prompt_parts.append("Seu saldo mensal não permite acumular para a reserva no momento.")


    full_prompt = "\n".join(prompt_parts)
    # print("\n--- PROMPT ENVIADO À IA ---\n", full_prompt, "\n----------------------------") # Descomentar para depurar o prompt

    try:
        response = MODEL.generate_content(full_prompt)
        return response.text
    except Exception as e:
        print(f"Erro ao chamar a IA Generativa: {e}")
        return "Não foi possível gerar uma análise detalhada da IA neste momento."

def analisar_fluxo_caixa(dados_usuario):
    """
    Calcula o total de gastos, o saldo mensal, determina o status do fluxo de caixa
    e calcula o total de gastos essenciais.
    """
    dados_usuario.total_gastos_mensais = sum(dados_usuario.gastos_por_categoria.values())

    # Calcula o total de gastos essenciais
    dados_usuario.total_gastos_essenciais = sum(
        valor for categoria, valor in dados_usuario.gastos_por_categoria.items()
        if dados_usuario.classificacao_gastos.get(categoria, {}).get('tipo') == 'Essencial'
    )

    dados_usuario.saldo_mensal = dados_usuario.renda_liquida_mensal - dados_usuario.total_gastos_mensais

    if dados_usuario.saldo_mensal > 0:
        dados_usuario.status_fluxo_caixa = "Superavitário"
    elif dados_usuario.saldo_mensal < 0:
        dados_usuario.status_fluxo_caixa = "Deficitário"
    else:
        dados_usuario.status_fluxo_caixa = "Equilibrado"

    print(f"\n--- Análise Numérica de Fluxo de Caixa Concluída ---")
    print(f"Total de Gastos Mensais: R$ {dados_usuario.total_gastos_mensais:.2f}")
    print(f"Total de Gastos Essenciais: R$ {dados_usuario.total_gastos_essenciais:.2f}")
    print(f"Saldo Mensal (Renda - Gastos): R$ {dados_usuario.saldo_mensal:.2f}")
    print(f"Status: {dados_usuario.status_fluxo_caixa}")


# --- Funções de Planejamento da Reserva (Etapa 4) ---

def planejar_reserva_emergencia(dados_usuario):
    """
    Define o valor ideal da reserva de emergência e estrutura uma estratégia de construção.
    """
    print("\n--- Planejamento da Reserva de Emergência ---")

    # 4.1. Definição do Valor Ideal
    meses_sugerido = 6
    if dados_usuario.estabilidade_financeira.lower() in ['autônomo', 'informal']:
        meses_sugerido = 12

    dados_usuario.meses_reserva_sugerido = meses_sugerido
    dados_usuario.valor_reserva_emergencia_ideal = dados_usuario.total_gastos_essenciais * meses_sugerido

    print(f"Valor ideal da sua reserva de emergência ({meses_sugerido} meses de gastos essenciais): R$ {dados_usuario.valor_reserva_emergencia_ideal:.2f}")

    # 4.2. Estratégia de Construção
    dados_usuario.saldo_mensal_para_reserva = max(0, dados_usuario.saldo_mensal)

    if dados_usuario.saldo_mensal_para_reserva > 0:
        dados_usuario.tempo_para_montar_reserva_meses = dados_usuario.valor_reserva_emergencia_ideal / dados_usuario.saldo_mensal_para_reserva
        dados_usuario.meta_mensal_reserva = dados_usuario.saldo_mensal_para_reserva

        print(f"Com um saldo mensal disponível de R$ {dados_usuario.saldo_mensal_para_reserva:.2f},")
        print(f"você levaria aproximadamente {dados_usuario.tempo_para_montar_reserva_meses:.1f} meses para montar a reserva.")
        print(f"Meta mensal sugerida para a reserva: R$ {dados_usuario.meta_mensal_reserva:.2f}")

    else:
        dados_usuario.tempo_para_montar_reserva_meses = float('inf')
        dados_usuario.meta_mensal_reserva = 0.0
        print("Seu saldo mensal não permite iniciar a reserva de emergência neste momento.")
        print("Foque primeiro em equilibrar suas finanças.")

    print("Planejamento da reserva concluído.")


# --- Funções de Acompanhamento e Reforço (Etapa 5) ---

def gerar_relatorio_mensal_simulado(dados_usuario):
    """
    Simula a geração de um relatório mensal em formato de texto.
    Em uma versão real, isso geraria um PDF ou painel interativo mais complexo.
    """
    print("\n--- Gerando Relatório Mensal Simulado ---")
    relatorio_parts = [
        "--- Relatório Mensal de Acompanhamento Financeiro ---",
        f"Período: Análise Inicial", # Simulado como análise inicial
        f"Status do Fluxo de Caixa: {dados_usuario.status_fluxo_caixa}",
        f"Saldo Mensal: R$ {dados_usuario.saldo_mensal:.2f}",
        "",
        "Resumo de Gastos:",
        f"Total Geral: R$ {dados_usuario.total_gastos_mensais:.2f}",
        f"Total Essenciais: R$ {dados_usuario.total_gastos_essenciais:.2f}",
        f"Total Supérfluos: R$ {dados_usuario.total_gastos_mensais - dados_usuario.total_gastos_essenciais:.2f}",
        "",
        "Situação da Reserva de Emergência:",
        f"Valor Ideal: R$ {dados_usuario.valor_reserva_emergencia_ideal:.2f}",
        f"Meta Mensal para Construção: R$ {dados_usuario.meta_mensal_reserva:.2f}",
        f"Tempo Estimado para Atingir: {dados_usuario.tempo_para_montar_reserva_meses:.1f} meses" if dados_usuario.tempo_para_montar_reserva_meses != float('inf') else "Saldo insuficiente para iniciar",
        "",
        "Análise e Recomendações da IA:",
        dados_usuario.feedback_ia_comportamento # Inclui o feedback gerado pela IA
    ]
    dados_usuario.relatorio_mensal_simulado = "\n".join(relatorio_parts)
    print("Relatório simulado gerado.")


def gerar_feedback_comportamento_ia(dados_usuario):
    """
    Invoca a IA Generativa para gerar feedback personalizado e recomendações de comportamento.
    Este feedback será incluído no relatório simulado e na análise geral.
    """
    # A análise principal da IA (gerar_analise_ia) já está sendo usada para isso.
    # Vamos apenas garantir que o resultado seja armazenado no campo correto.
    dados_usuario.feedback_ia_comportamento = gerar_analise_ia(dados_usuario)
    print("Feedback de comportamento da IA gerado.")


# --- Funções de Geração de Planilha (Etapa 3) ---

def gerar_planilha_google_sheets(dados_usuario, sheet_name="Meu_Painel_Financeiro_IA"):
    """
    Cria ou atualiza uma planilha Google Sheets com os dados financeiros do usuário,
    incluindo o planejamento da reserva e resumo do relatório simulado.
    Compartilha com o usuário pessoal.
    Retorna a URL da planilha ou None em caso de erro.
    """
    print(f"\n--- Tentando gerar/atualizar Painel Interativo no Google Sheets ({sheet_name}) ---")
    spreadsheet = None
    try:
        # Tenta abrir a planilha existente
        try:
            spreadsheet = CLIENT.open(sheet_name)
            print(f"Planilha '{sheet_name}' encontrada. Atualizando...")
        except gspread.exceptions.SpreadsheetNotFound:
            # Se não encontrar, tenta criar uma nova planilha
            print(f"Planilha '{sheet_name}' não encontrada. Criando nova planilha...")
            try:
                spreadsheet = CLIENT.create(sheet_name)
                print(f"Planilha '{sheet_name}' criada com sucesso.")
                # O email da conta de serviço já tem acesso por ser o criador/proprietário.
                # Agora precisamos compartilhar com o email pessoal do usuário.

            except Exception as create_error:
                print(f"ERRO GRAVE: Falha ao criar a planilha '{sheet_name}'.")
                print(f"Detalhes do erro de criação: {create_error}")
                print("Causas comuns: Permissões insuficientes da conta de serviço no Google Drive para criar arquivos.")
                return None # Retorna None se a criação falhar

        # Se chegamos aqui, a planilha foi aberta ou criada com sucesso (spreadsheet is not None)

        # ----- Compartilhar a planilha com o usuário pessoal -----
        if dados_usuario.email_pessoal_usuario:
            try:
                # Compartilha com o email pessoal do usuário com permissão de escrita
                spreadsheet.share(dados_usuario.email_pessoal_usuario, perm_type='user', role='writer')
                print(f"Planilha compartilhada com sucesso com {dados_usuario.email_pessoal_usuario}.")
            except Exception as share_error:
                print(f"AVISO: Não foi possível compartilhar a planilha com {dados_usuario.email_pessoal_usuario}.")
                print(f"Detalhes do erro de compartilhamento: {share_error}")
                print("Causas comuns: Email inválido, permissões insuficientes da conta de serviço para compartilhar.")
        else:
            print("AVISO: Email pessoal do usuário não fornecido. A planilha não será compartilhada automaticamente.")


        # ----- Aba de Resumo (Atualizada para incluir Reserva e Resumo Relatório) -----
        try:
            worksheet_resumo = spreadsheet.worksheet("Resumo Geral")
        except gspread.exceptions.WorksheetNotFound:
            worksheet_resumo = spreadsheet.add_worksheet(title="Resumo Geral", rows="200", cols="20") # Aumentado linhas

        worksheet_resumo.clear()

        resumo_data = [
            ["Item", "Valor/Detalhe"],
            ["Renda Líquida Mensal", dados_usuario.renda_liquida_mensal],
            ["Total de Gastos Mensais", dados_usuario.total_gastos_mensais],
            ["Total de Gastos Essenciais", dados_usuario.total_gastos_essenciais],
            ["Saldo Mensal", dados_usuario.saldo_mensal],
            ["Status do Fluxo de Caixa", dados_usuario.status_fluxo_caixa],
            ["", ""],
            ["--- Reserva de Emergência ---", ""],
            ["Valor Ideal da Reserva", dados_usuario.valor_reserva_emergencia_ideal],
            [f"({dados_usuario.meses_reserva_sugerido} meses de gastos essenciais)", ""],
            ["Saldo Mensal para Reserva", dados_usuario.saldo_mensal_para_reserva],
            ["Tempo Estimado para Montar (meses)", dados_usuario.tempo_para_montar_reserva_meses if dados_usuario.tempo_para_montar_reserva_meses != float('inf') else "Saldo insuficiente"],
            ["Meta Mensal Sugerida", dados_usuario.meta_mensal_reserva],
            ["", ""],
            ["--- Relatório Mensal Simulado ---", ""], # Adicionado cabeçalho
            [dados_usuario.relatorio_mensal_simulado], # Conteúdo do relatório simulado
            ["", ""],
            #["--- Análise e Recomendações da IA (Comportamento) ---", ""], # Adicionado cabeçalho
            #[dados_usuario.feedback_ia_comportamento] # Feedback da IA
        ]
        worksheet_resumo.update('A1', resumo_data)

        # Formatação básica para valores monetários e negrito
        worksheet_resumo.format('B2:B5', {'numberFormat': {'type': 'CURRENCY', 'pattern': 'R$#,##0.00'}})
        worksheet_resumo.format('B9:B11', {'numberFormat': {'type': 'CURRENCY', 'pattern': 'R$#,##0.00'}})
        worksheet_resumo.format('A1:B1', {'textFormat': {'bold': True}})
        worksheet_resumo.format('A8:B8', {'textFormat': {'bold': True}})
        worksheet_resumo.format('A15:B15', {'textFormat': {'bold': True}}) # Cabeçalho Relatório
        worksheet_resumo.format('A18:B18', {'textFormat': {'bold': True}}) # Cabeçalho Análise IA

        print("Aba 'Resumo Geral' atualizada com dados da análise, reserva e relatório simulado.")

        # ----- Aba de Gastos Detalhados (Mantida) -----
        try:
            worksheet_gastos = spreadsheet.worksheet("Gastos Detalhados")
        except gspread.exceptions.WorksheetNotFound:
            worksheet_gastos = spreadsheet.add_worksheet(title="Gastos Detalhados", rows="100", cols="10")

        worksheet_gastos.clear()

        gastos_header = ["Categoria", "Valor (R$)", "Tipo", "Natureza"]
        gastos_rows = [gastos_header]
        for categoria, valor in dados_usuario.gastos_por_categoria.items():
            classif = dados_usuario.classificacao_gastos.get(categoria, {'tipo': 'N/A', 'natureza': 'N/A'})
            gastos_rows.append([
                categoria,
                valor,
                classif.get('tipo', 'N/A'),
                classif.get('natureza', 'N/A')
            ])
        worksheet_gastos.update('A1', gastos_rows)
        worksheet_gastos.format('B2:B', {'numberFormat': {'type': 'CURRENCY', 'pattern': 'R$#,##0.00'}})
        worksheet_gastos.format('A1:D1', {'textFormat': {'bold': True}})
        print("Aba 'Gastos Detalhados' atualizada.")

        # ----- Aba de Dívidas (Mantida) -----
        try:
            worksheet_dividas = spreadsheet.worksheet("Dívidas")
        except gspread.exceptions.WorksheetNotFound:
            worksheet_dividas = spreadsheet.add_worksheet(title="Dívidas", rows="100", cols="10")

        worksheet_dividas.clear()

        dividas_header = ["Tipo", "Valor Original (R$)", "Valor Restante (R$)", "Taxa Juros Anual (%)", "Parcelas Totais", "Parcelas Restantes"]
        dividas_rows = [dividas_header]
        for divida in dados_usuario.dividas:
            dividas_rows.append([
                divida['tipo'],
                divida['valor_original'],
                divida['valor_restante'],
                divida['taxa_juros_anual'] * 100,
                divida['parcelas_totais'],
                divida['parcelas_restantes']
            ])
        if len(dividas_rows) == 1:
            dividas_rows.append(["Nenhuma dívida informada.", "", "", "", "", ""])

        worksheet_dividas.update('A1', dividas_rows)
        worksheet_dividas.format('B2:C', {'numberFormat': {'type': 'CURRENCY', 'pattern': 'R$#,##0.00'}})
        worksheet_dividas.format('D2:D', {'numberFormat': {'type': 'PERCENT', 'pattern': '0.00%'}})
        worksheet_dividas.format('A1:F1', {'textFormat': {'bold': True}})
        print("Aba 'Dívidas' atualizada.")

        # Retorna a URL da planilha se tudo deu certo
        return spreadsheet.url

    except Exception as e:
        print(f"ERRO GERAL ao gerar/atualizar a planilha Google Sheets: {e}")
        print("Verifique as permissões da sua conta de serviço no Google Cloud Console.")
        return None


# --- Fluxo Principal (Incluindo Etapa 5) ---

def iniciar_processo_financeiro():
    """Inicia o processo completo de coleta, análise, planejamento de reserva, acompanhamento e geração de planilha."""
    print("---🚀 Bem-vindo ao seu Agente Financeiro Pessoal com IA 💰---")
    print ("\n")
    print("Vamos coletar informações, analisar, planejar sua reserva, gerar um relatório e painel.")

    dados_usuario = DadosFinanceirosUsuario()

    # Coleta o email pessoal do usuário logo no início
    dados_usuario.email_pessoal_usuario = get_string_input("Por favor, digite seu email pessoal do Google (para compartilhar a planilha): ")

    # Etapa 1: Coleta de Dados
    print("\n--- ETAPA 1: Diagnóstico Financeiro Inicial 📊 ---")
    print ("\n")
    dados_usuario.renda_liquida_mensal = get_float_input("Qual a sua renda líquida mensal (após impostos)? R$ ")
    dados_usuario.fonte_renda = get_string_input("Qual a sua principal profissão/fonte de renda? ")

    gastos_coletados, classificacao_map_coletada = coletar_gastos_por_categoria()
    dados_usuario.gastos_por_categoria = gastos_coletados
    dados_usuario.classificacao_gastos = classificacao_map_coletada

    dados_usuario.patrimonio = coletar_patrimonio()
    dados_usuario.dividas = coletar_dividas()
    dados_usuario.dependentes = get_int_input("Quantos dependentes você possui? ")
    dados_usuario.estabilidade_financeira = get_string_input("Qual o seu grau de estabilidade financeira (Ex: CLT, Autônomo, Informal)? ")

    print("\nColeta de dados da Etapa 1 concluída.")

    # Etapa 2: Análise de Fluxo de Caixa (Numérica)
    print("\n--- ETAPA 2: Análise de Fluxo de Caixa e Saldos (Numérica) 📈---")
    print ("\n")
    analisar_fluxo_caixa(dados_usuario) # Realiza os cálculos numéricos e classifica gastos

    # Etapa 4: Planejamento da Reserva de Emergência
    print("\n--- ETAPA 3: Planejamento da Reserva de Emergência 🗺️---")
    print ("\n")
    planejar_reserva_emergencia(dados_usuario)

    # Etapa 5: Acompanhamento e Reforço de Comportamento Financeiro
    print("\n--- ETAPA 4: Acompanhamento e Reforço de Comportamento Financeiro 📒---")
    print ("\n")
    # Primeiro, gera o feedback da IA com base em todos os dados e planejamento
    gerar_feedback_comportamento_ia(dados_usuario)
    # Depois, gera o relatório simulado que inclui o feedback da IA
    gerar_relatorio_mensal_simulado(dados_usuario)


    # Exibe o resumo completo no console após todas as análises
    print("\n--- Resumo Completo das Análises, Planejamento e Relatório ---")
    print(dados_usuario)

    # Etapa 3: Geração do Painel no Google Sheets (Chamada DEPOIS de todas as análises e planejamento)
    # O painel agora incluirá os dados da reserva e o resumo do relatório/feedback.
    print("\n--- ETAPA 5: Criação de Planilha e Monitoramento Personalizado ---")
    print ("\n")
    planilha_url = gerar_planilha_google_sheets(dados_usuario, sheet_name="Meu_Painel_Financeiro_Pessoal_IA")
    if planilha_url:
        print(f"\n✅ Processo Concluído! Painel Interativo no Google Sheets Gerado! Acesse em: {planilha_url}")
        print(f"Verifique seu email ({dados_usuario.email_pessoal_usuario}) para o convite de compartilhamento (se for a primeira vez).")
    else:
        print("\n❌ Processo Concluído com Erros. Não foi possível gerar o painel no Google Sheets. Verifique os erros acima.")

    return dados_usuario

# --- Execução ---
if __name__ == "__main__":
    #print("--- Configuração Inicial Necessária ---")
    #print("1. Execute a célula: from google.colab import drive; drive.mount('/content/drive')")
    #print(f"2. ATUALIZE A VARIÁVEL 'SERVICE_ACCOUNT_KEY_FILE' no código com o caminho COMPLETO para o seu arquivo JSON no Google Drive.")
    #print(f"   Caminho atual configurado: '{SERVICE_ACCOUNT_KEY_FILE}'")
    #print("3. Certifique-se de que a conta de serviço (do arquivo JSON) tem permissão de 'Editor' (ou superior) no Google Cloud Console.")
    #print("---------------------------------------")

    # Inicia o processo principal
    dados_completos = iniciar_processo_financeiro()
    print("\nFluxo completo do Agente Financeiro Pessoal concluído.")


---🚀 Bem-vindo ao seu Agente Financeiro Pessoal com IA 💰---


Vamos coletar informações, analisar, planejar sua reserva, gerar um relatório e painel.
Por favor, digite seu email pessoal do Google (para compartilhar a planilha): arthur.scosta1355@gmail.com

--- ETAPA 1: Diagnóstico Financeiro Inicial 📊 ---


Qual a sua renda líquida mensal (após impostos)? R$ 1500
Qual a sua principal profissão/fonte de renda? Analisa de Power BI

--- Gastos Mensais por Categoria ---
Digite o valor gasto para cada categoria. Digite 'fim' quando terminar.
Gasto com Moradia (Aluguel) (R$): 500
Gasto com Alimentação (R$): 120
Gasto com Transporte (R$): 150
Gasto com Saúde (R$): 0
Gasto com Educação (R$): 122
Gasto com Lazer (R$): 503
Gasto com Assinaturas (R$): 20
Gasto com Contas de Consumo (água, luz, gás) (R$): 300
Adicionar outra categoria de gasto? (ou 'fim'): Cabelo
Classifique o gasto 'Cabelo':
É 'Essencial' ou 'Supérfluo'? Essencial
É 'Fixo' ou 'Variável'? Fixo
Gasto com Cabelo (R$): 70
Adicionar o